# ADK Tools: Built-in, Function, Third-party, MCP & OpenAPI (with diagrams & examples)

This module explains **how tools work in Google ADK**, the **available tool types**, when to use which, their **limitations**, and gives runnable code templates.

**What is a tool?** In ADK, a *Tool* is a capability an agent can call to act beyond text generation (web search, code execution, database queries, external APIs, even other agents). :contentReference[oaicite:0]{index=0}

**Tool families covered:**
- Function Tools (incl. Long-running & Agent-as-Tool) :contentReference[oaicite:1]{index=1}
- Built-in Tools: Google Search, Code Execution, Vertex AI RAG, Vertex AI Search, BigQuery :contentReference[oaicite:2]{index=2}
- Third-party Tools (LangChain, CrewAI wrappers) :contentReference[oaicite:3]{index=3}
- Google Cloud Tools (Apigee / App Integration / DB toolbox) :contentReference[oaicite:4]{index=4}
- MCP Tools (consume or expose MCP) :contentReference[oaicite:5]{index=5}
- OpenAPI Tools (auto-generate REST tools from spec) :contentReference[oaicite:6]{index=6}

> **Latest constraints you must know (Sept 2025)**
> - **Built-in tools** (like `google_search` and built-in code execution) require **Gemini 2** models and have composition limits (details below). :contentReference[oaicite:7]{index=7}
> - ADK **parallelizes function tools** (v1.10.0+), so write async tools to benefit. :contentReference[oaicite:8]{index=8}


## Big picture: how an ADK agent decides to call tools

```mermaid
flowchart TD
    A[User message] --> B[LlmAgent - planner]
    B -->|No tool needed| R[Respond with text]
    B -->|Tool call needed| C{Which tool family?}
    C --> D[Built-in tool<br/>Google Search, Code Exec,<br/>Vertex RAG/Search, BigQuery]
    C --> E[Function tool<br/>your Python function]
    C --> F[Agent-as-Tool<br/>wrap specialized agent]
    C --> G[Third-party wrapper<br/>LangChain/CrewAI]
    C --> H[MCP toolset<br/>connect MCP server]
    C --> I[OpenAPI toolset<br/>generate REST tools]
    D --> J[Runner executes tool]
    E --> J
    F --> J
    G --> J
    H --> J
    I --> J
    J --> K[LLM summarizes results]
    K --> L[Final user response]
    
    style A fill:#e1f5fe
    style B fill:#f3e5f5
    style C fill:#fff3e0
    style R fill:#e8f5e8
    style L fill:#e8f5e8
```


## 1) Function Tools (your Python → a tool)
Function tools are the simplest way to add custom capabilities. ADK inspects your function signature, docstring, and types to build the calling schema the LLM uses. Supports required/optional params, `typing.Optional`, and variadics are ignored for schema. Prefer dict return values. 

```python
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

def convert_temperature(value: float, unit_from: str, unit_to: str = "C") -> dict:
    """Convert temperatures. Args: value; unit_from in ['C','F']; unit_to in ['C','F'] (optional, default 'C'). Returns dict."""
    if unit_from == unit_to:



        return {"status": "ok", "result": value}
    if unit_from == "C" and unit_to == "F":
        return {"status": "ok", "result": (value * 9/5) + 32}
    if unit_from == "F" and unit_to == "C":
        return {"status": "ok", "result": (value - 32) * 5/9}
    return {"status": "error", "error_message": "Unsupported units"}

agent = LlmAgent(
    model="gemini-2.0-flash",
    name="unit_helper",
    instruction=(
        "Use the convert_temperature tool for conversions. If tool returns status=error, explain why."
    ),
    tools=[convert_temperature],  # ADK wraps it as a FunctionTool automatically
)
```

### Agent-as-a-Tool
Wrap a **specialized agent** as a callable tool using `AgentTool`, letting a parent agent delegate but retain control (vs sub-agents which transfer control). Import via `google.adk.tools.agent_tool.AgentTool`. :contentReference[oaicite:11]{index=11}

```python
from google.adk.tools.agent_tool import AgentTool

web_researcher = LlmAgent(model="gemini-2.0-flash", name="WebResearcher", instruction="Search + summarize on a topic.")
math_solver = LlmAgent(model="gemini-2.0-flash", name="MathSolver", instruction="Do accurate calculations.")

root = LlmAgent(
  model="gemini-2.0-flash",
  name="RootAgent",
  instruction="Use the specialized agents as tools as needed.",
  tools=[AgentTool(web_researcher), AgentTool(math_solver)]
)
```


## 2) Built-in Tools (Google Search, Code Execution, Vertex RAG/Search, BigQuery)

**Compatibility & limits:**
- **Use Gemini 2 models** (e.g., `gemini-2.0-flash`) for built-ins like Google Search and built-in Code Execution. :contentReference[oaicite:12]{index=12}
- **One built-in tool per (root or single) agent**; you **cannot** mix a built-in tool with other tools in the same agent, and **cannot** use built-ins inside sub-agents. Work around by wrapping specialized agents with `AgentTool`. :contentReference[oaicite:13]{index=13}
- When using **Google Search grounding**, if suggestions are returned, you **must display them** (HTML comes in `renderedContent`).

```mermaid
flowchart LR
    A[Need Search + Code Exec] --> B{One agent?}
    B -->|yes| X[Not supported with two built-ins]
    B -->|no| C[Make SearchAgent<br/>google_search]
    B -->|no| D[Make CodeAgent<br/>BuiltInCodeExecutor]
    C --> E[Wrap with AgentTool]
    D --> E
    E --> F[RootAgent uses both tools<br/>via AgentTool]
    
    style A fill:#e1f5fe
    style B fill:#fff3e0
    style X fill:#ffebee
    style C fill:#e8f5e8
    style D fill:#e8f5e8
    style E fill:#f3e5f5
    style F fill:#e3f2fd
```


### Google Search (grounding)
```python
from google.adk.agents import LlmAgent
from google.adk.tools import google_search

search_agent = LlmAgent(
    model="gemini-2.0-flash",  # Gemini 2 required
    name="Search",
    instruction=(
        "Use the google_search tool before answering. If response contains renderedContent suggestions, surface them."
    ),
    tools=[google_search],
)
```
Learn more about grounding policy & behavior. :contentReference[oaicite:15]{index=15}

### Built-in Code Execution
```python
from google.adk.agents import LlmAgent
from google.adk.code_executors import BuiltInCodeExecutor

code_agent = LlmAgent(
    model="gemini-2.0-flash",  # Gemini 2 required for built-in executor
    name="Calculator",
    code_executor=BuiltInCodeExecutor(),
    instruction=(
        "Write and execute short Python to compute answers. Return just the number."
    ),
)
```

## 3) Third-party Tools (LangChain & CrewAI)
ADK provides wrappers to reuse tools from popular ecosystems. Example below shows LangChain Tavily search via `LangchainTool`. :contentReference[oaicite:18]{index=18}


In [ ]:
# LangChain Tavily example
!pip -q install langchain_community tavily-python
from google.adk.agents import LlmAgent
from google.adk.tools.langchain_tool import LangchainTool
from langchain_community.tools import TavilySearchResults
import os

os.environ.setdefault("TAVILY_API_KEY", "<YOUR_KEY>")
tavily = TavilySearchResults(max_results=5, search_depth="advanced", include_answer=True)
adk_tavily = LangchainTool(tool=tavily)

lc_agent = LlmAgent(
  model="gemini-2.0-flash",
  name="LCSearch",
  instruction="Use Tavily for realtime web context before answering.",
  tools=[adk_tavily]
)

## 4) MCP Tools (Model Context Protocol)
Connect ADK agents to **MCP servers** (e.g., filesystem, Google Maps) using `MCPToolset`. Use `StdioConnectionParams` (local `npx` servers) or SSE/HTTP for remote. :contentReference[oaicite:19]{index=19}


In [ ]:
# Minimal MCP filesystem server example (local development)
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters
import os

target_folder = os.path.abspath(".")  # pick a safe folder
mcp_tools = MCPToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=["-y", "@modelcontextprotocol/server-filesystem", target_folder],
        )
    )
)

mcp_agent = LlmAgent(
  model="gemini-2.0-flash",
  name="FileAssistant",
  instruction="Use MCP tools to list/read files as requested.",
  tools=[mcp_tools]
)